# OAuth2 Client Credentials Demo

This notebook demonstrates running the ETL pipeline against an API endpoint using OAuth2 Client Credentials Grant authentication via Keycloak.

Unlike Password Grant, Client Credentials authenticates the application itself, not on behalf of a user. This is the recommended flow for service-to-service communication.

## Prerequisites

Start Keycloak and the mock API service:
```bash
make up-keycloak
```

Services:
- Keycloak Admin Console: `http://localhost:8180` (admin/admin)
- Mock API: `http://mock-api:8000` (from Docker network)

## Client Credentials

- Client ID: `etl-client`
- Client Secret: `etl-client-secret`

In [1]:
import sys
from pathlib import Path

In [8]:
# project_root = Path("/opt/spark/work")
project_root = Path("/opt/spark/app")
src_path = project_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sha2, expr
from pipeline.orchestrator import run_pipeline

In [10]:
spark = (
    SparkSession.builder
    .appName("oauth2_client_credentials_demo_pipeline")
    .getOrCreate()
)

## Create Source DataFrame

Generate a DataFrame with unique tracking IDs that will be used to make API requests.

In [11]:
df = (
    spark.range(50)
         .repartition(4)
         .select(
             sha2(expr("uuid()"), 256).alias("tracking_id")
         )
)
df.show(10)

+--------------------+
|         tracking_id|
+--------------------+
|7c04fe275399d5510...|
|c5127aadedecfc274...|
|b67ad3b6f5ad1e554...|
|7c3bb7899bb40a67f...|
|d58efe410abb69d13...|
|2f9bd00fb48a41f6c...|
|f38364c88fc1dd8b9...|
|5dbe37d4911423343...|
|47203c709ac0bd857...|
|7d956720afe7799ea...|
+--------------------+
only showing top 10 rows



## Run Pipeline

Execute the ETL pipeline using the OAuth2 Client Credentials configuration.

The pipeline will:
1. Authenticate with Keycloak using client credentials (no user involved)
2. Distribute the access token to all Spark executors
3. Make API requests with `Authorization: Bearer <token>` header
4. Automatically refresh tokens when they expire

In [12]:
config_path = project_root / "configs" / "examples" / "oauth2_client_credentials_demo.yml"

In [13]:
run_pipeline(
    spark=spark,
    config_path=config_path,
    source_df=df,
    source_id="tracking_id"
)

2026-02-10 14:01:54,561 [INFO] [PipelineOrchestrator]: Starting driver-side authentication runtime service
2026-02-10 14:01:54,562 [INFO] [RpcBootstrapper]: Starting RPC token service...
2026-02-10 14:01:54,597 [INFO] AsyncBackgroundService[DriverTokenManager]: Background service started
2026-02-10 14:01:54,608 [INFO] AsyncBackgroundService[RpcService]: Background service started
2026-02-10 14:01:54,609 [INFO] [TokenRpcService]: Started at http://3d452d1cc6a1:38383
2026-02-10 14:01:54,610 [INFO] [RpcBootstrapper]: TokenManager background refresh started.
2026-02-10 14:01:54,610 [INFO] [RpcBootstrapper]: RPC Token Service running at http://3d452d1cc6a1:38383
2026-02-10 14:01:54,610 [INFO] [PipelineOrchestrator]: Adding authentication middleware
2026-02-10 14:01:54,611 [INFO] [PipelineOrchestrator]: ====== Executing Pipeline ======
2026-02-10 14:01:54,611 [INFO] [PipelineOrchestrator]: Request from URL: /http://mock-api:8000/api/oauth2/data
2026-02-10 14:01:54,612 [INFO] [TableManager]: 

## Verify Results

Read the sink table to verify the API responses were captured.

In [14]:
response_df = spark.table("demo.oauth2_client_credentials_demo_response")
response_df.show(10)

+--------------------+--------------------+--------------------+------+--------------------+--------------+--------------------+-----------+--------------------+--------------------+-------+-------------+--------+--------------------+--------------------+
|          request_id|            row_hash|                 url|method|     request_headers|request_params|    request_metadata|status_code|    response_headers|           body_text|success|error_message|attempts|   response_metadata|       _request_time|
+--------------------+--------------------+--------------------+------+--------------------+--------------+--------------------+-----------+--------------------+--------------------+-------+-------------+--------+--------------------+--------------------+
|2f9bd00fb48a41f6c...|4810533a8f1fa8309...|/http://mock-api:...|   GET|{"Accept": "appli...|            {}|{"vendor": "mock-...|        200|{"Date": "Tue, 10...|{"request_id":"c8...|   true|         NULL|       1|{"connection_warm..

In [15]:
# Summary statistics
print(f"Total records: {response_df.count()}")
response_df.groupBy("status_code").count().show()

Total records: 50
+-----------+-----+
|status_code|count|
+-----------+-----+
|        200|   50|
+-----------+-----+



In [16]:
# Sample response body - note the auth_method field shows "oauth2:bearer"
response_df.select("body_text").limit(3).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|body_text                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------